In [38]:
from datasets import load_dataset
import pandas as pd


In [39]:
ds = load_dataset("Amod/mental_health_counseling_conversations", split='train')

In [44]:
first_1000 = ds.select(range(1000))
df_first_1000 = first_1000.to_pandas()

df_first_1000

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...
...,...,...
995,"I was the one who ended it, and I'm so glad I ...","Your challenge is called ""co-dependency."" Peop..."
996,I was a victim of an incident. I've been made ...,"As you seemed to have learned, the impact will..."
997,I was a victim of an incident. I've been made ...,I'm glad you have some time when you're not th...
998,"I know I was molested as a child, but I have n...",I'm not sure that I have the answer to your qu...
